In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

In [ ]:
train.head()

In [ ]:
train.to_numpy().shape

In [ ]:
test.to_numpy().shape

In [ ]:
np.max(train.to_numpy()[0])

In [ ]:
train_image = train.to_numpy()[0:,1:].reshape(42000,28,28,1)
test_image = test.to_numpy().reshape(28000,28,28,1)

In [ ]:
train_image = train_image.astype(float) / 255.0
test_image = test_image.astype(float) / 255.0

In [ ]:
img = train_image[0]
print(img.shape)
plt.imshow(img.squeeze())

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import InputLayer, Conv2D, LeakyReLU, Conv2DTranspose
from keras.losses import MeanSquaredError
from keras.optimizers import Adam

In [ ]:
class AutoEncoder(keras.Model):
    def __init__(self,input_shape):
        super(AutoEncoder,self).__init__()
        #encoder
        self.encoder = keras.Sequential([
            InputLayer(input_shape=input_shape),
            Conv2D(64,kernel_size=3,strides=2,padding='same'),
            LeakyReLU(0.001),
            Conv2D(64,kernel_size=3,strides=2,padding='same'),
            LeakyReLU(0.001)
        ])

        #decoder
        self.decoder = keras.Sequential([
            Conv2DTranspose(64,kernel_size=3,strides=2,padding='same'),
            LeakyReLU(0.001),
            Conv2DTranspose(64,kernel_size=3,strides=2,padding='same'),
            LeakyReLU(0.001),
            Conv2D(input_shape[2],kernel_size=3,padding='same'),
            LeakyReLU(0.001)
        ])

    def call(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
ae = AutoEncoder((28,28,1))


In [ ]:
opt = Adam(0.0001)
loss = MeanSquaredError()

In [ ]:
ae.compile(optimizer = opt,loss = loss)

In [ ]:
history = ae.fit(train_image,train_image,batch_size=128,epochs=10,verbose=1)

In [ ]:
ae.summary()

In [ ]:

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    encoded = ae.encoder(tf.expand_dims(test_image[i],axis=0))
    cleaned = ae.decoder(encoded)
    ax = plt.subplot(2, n, i + 1)
    plt.title("original")
    plt.imshow(tf.squeeze(test_image[i]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(tf.squeeze(cleaned))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='lower right')